In [27]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [28]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/ppp_geo_fips23.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,5502308207,AK,"5015 BUSINESS PARK BLVD, ANCHORAGE, AK",61.174949,-149.888309,2.020002e+13
1,6110847106,AK,"7000 Uula St, BARROW, AK",71.297290,-156.728897,2.185000e+13
2,4539098204,AK,"4700 Old International Airport Rd, Anchorage, AK",61.173232,-149.972718,2.020002e+13
3,5120868804,AK,"540 W Intl Airport Rd, Anchorage, AK",61.173164,-149.893283,2.020003e+13
4,6650277102,AK,"4300 BARTLETT STREET, HOMER, AK",59.652704,-151.549793,2.122001e+13


In [31]:
pd.set_option('display.max_columns', None)
#Latitude, Longitude included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
392,2725348408,AK,"525 Second Street, Skagway, AK",40.670782,-73.976908,NaN
1095,4188937110,AK,"525 SECOND STREET, SKAGWAY, AK",40.670782,-73.976908,NaN
1601,9864858410,AK,"525 Second Street, Skagway, AK",40.670782,-73.976908,NaN
2874,6205747005,AL,"1980 hwy 315, SAGINAW, AL",34.212860,-89.696248,NaN
3234,5260357102,AL,"5015 Business Park BLVD, NEW MARKET, AL",61.174949,-149.888309,NaN
...,...,...,...,...,...,...
965547,9725917702,Not Available,"17 S High St., COLUMBUS, OH",39.961717,-83.001011,NaN
965548,9666867710,Not Available,"17 S High St., COLUMBUS, OH",39.961717,-83.001011,NaN
965549,9662697710,Not Available,"17 S High St., COLUMBUS, OH",39.961717,-83.001011,NaN
965550,9724667708,Not Available,"17 S High St., COLUMBUS, OH",39.961717,-83.001011,NaN


In [30]:
fips_check_scraped_null_il = fips_check_scraped_null.loc[fips_check_scraped_null["BorrowerState"]=="IL"]
fips_check_scraped_null_il

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
309462,4357417000,IL,"18960 County Hwy 5 W, EAST DUBUQUE, IL",42.508105,-90.565332,NaN
310110,6227707103,IL,"22069 US HIGHWAY 34, PRINCETON, IL",40.392967,-104.634462,NaN
310134,4428188509,IL,"22069 US Highway 34, Princeton, IL",40.392967,-104.634462,NaN
310220,3984847201,IL,"14 N 845 US-20, Hampshire, IL",44.370091,-108.038415,NaN
311283,2252087100,IL,"14 N 850 Route 20, HAMPSHIRE, IL",42.468637,-73.419886,NaN
311327,3313488304,IL,"43451 N US Highway 41, Zion, IL",35.242488,-85.690479,NaN
311689,6478417409,IL,"1501 W. MAIN ST PO Drawer E, FAIRFIELD, IL",38.033561,-78.497707,NaN
311761,6829277004,IL,"43451 US HIGHWAY 41, ZION, IL",47.046908,-88.518619,NaN
311943,6463967004,IL,"11600 Sterling Parkway, RICHMOND, IL",42.495414,-88.320057,NaN
311974,7639977104,IL,"7505 S Us Highway 66, GARDNER, IL",36.145096,-96.002364,NaN


Manually Updated File Read-IN

In [38]:
data_xls = pd.read_excel('../../../data/null/il_fips_update.xlsx', dtype=str, index_col=None)
data_xls.to_csv('../../../data/null/il_fips_update.csv', encoding='utf-8', index=False)
il_update_df = pd.read_csv("../../../data/null/il_fips_update.csv")
il_update_df.index = il_update_df.index.astype('int64')
il_update_df

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,4357417000,IL,"18950 Rte 5 W, East Dubuque, IL",42.508094,-90.597843,NaN
1,6227707103,IL,"22069 US-34, Princeton, IL",41.406474,-89.430516,NaN
2,4428188509,IL,"22069 US-34, Princeton, IL",41.406474,-89.430516,NaN
3,3984847201,IL,"14N845 US-20, Hampshire, IL",42.080135,-88.449376,NaN
4,2252087100,IL,"14N850 US-20, Hampshire, IL",42.077661,-88.450393,NaN
5,3313488304,IL,"43451 US Hwy 41, Zion, IL",42.494413,-87.951154,NaN
6,6478417409,IL,"1501 W Main St, Fairfield, IL",38.380251,-88.382691,NaN
7,6829277004,IL,"43451 US Hwy 41, Zion, IL",42.494413,-87.951154,NaN
8,6463967004,IL,"11600 Sterling Pkwy, Richmond, IL",42.493819,-88.320251,NaN
9,7639977104,IL,"7505 Old Rte 66, Gardner, IL",41.177952,-88.322549,NaN


<h4>FIPS Script

In [39]:
lat = il_update_df["Lat"].astype(float).astype(str).to_list()
long = il_update_df["Long"].astype(float).astype(str).to_list()

In [40]:
#if FIPS_z file 
#lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
#long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [41]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            #startswith = [s for s in result_digits if s.startswith('51')] #VA
            fips_elements =[element for element in result_digits if len(element)==15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/30 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

Length Check

In [43]:
geo_id_list =geo_id 
print(len(geo_id_list))
print(geo_id_list)

30
[['550439612002038'], ['170119649001093'], ['170119649001093'], ['170898507102006'], ['170898507112014'], ['170978660002005'], ['171919552002005'], ['170978660002005'], ['171118701041004'], ['170630009002085'], ['171118715011024'], ['261439706002023'], ['170938901031004'], ['170898545041040'], ['170898545041040'], ['172010040041013'], ['180179513003007'], ['170938905013050'], ['170938905013050'], ['170898545091007'], ['170898527003029'], ['060590524102006'], ['180890427043017'], ['170938901071015'], ['170898524034064'], ['170938904021008'], ['170190104003020'], ['170318255051000'], ['171659551003028'], ['170898524034064']]


Matching FIPS to DataFrame

In [44]:
fips_fill = il_update_df

In [45]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,4357417000,IL,"18950 Rte 5 W, East Dubuque, IL",42.508094,-90.597843,[550439612002038]
1,6227707103,IL,"22069 US-34, Princeton, IL",41.406474,-89.430516,[170119649001093]
2,4428188509,IL,"22069 US-34, Princeton, IL",41.406474,-89.430516,[170119649001093]
3,3984847201,IL,"14N845 US-20, Hampshire, IL",42.080135,-88.449376,[170898507102006]
4,2252087100,IL,"14N850 US-20, Hampshire, IL",42.077661,-88.450393,[170898507112014]
5,3313488304,IL,"43451 US Hwy 41, Zion, IL",42.494413,-87.951154,[170978660002005]
6,6478417409,IL,"1501 W Main St, Fairfield, IL",38.380251,-88.382691,[171919552002005]
7,6829277004,IL,"43451 US Hwy 41, Zion, IL",42.494413,-87.951154,[170978660002005]
8,6463967004,IL,"11600 Sterling Pkwy, Richmond, IL",42.493819,-88.320251,[171118701041004]
9,7639977104,IL,"7505 Old Rte 66, Gardner, IL",41.177952,-88.322549,[170630009002085]


Remove brackets

In [46]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,4357417000,IL,"18950 Rte 5 W, East Dubuque, IL",42.508094,-90.597843,550439612002038
1,6227707103,IL,"22069 US-34, Princeton, IL",41.406474,-89.430516,170119649001093
2,4428188509,IL,"22069 US-34, Princeton, IL",41.406474,-89.430516,170119649001093
3,3984847201,IL,"14N845 US-20, Hampshire, IL",42.080135,-88.449376,170898507102006
4,2252087100,IL,"14N850 US-20, Hampshire, IL",42.077661,-88.450393,170898507112014
5,3313488304,IL,"43451 US Hwy 41, Zion, IL",42.494413,-87.951154,170978660002005
6,6478417409,IL,"1501 W Main St, Fairfield, IL",38.380251,-88.382691,171919552002005
7,6829277004,IL,"43451 US Hwy 41, Zion, IL",42.494413,-87.951154,170978660002005
8,6463967004,IL,"11600 Sterling Pkwy, Richmond, IL",42.493819,-88.320251,171118701041004
9,7639977104,IL,"7505 Old Rte 66, Gardner, IL",41.177952,-88.322549,170630009002085


State Adjustment

fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '05', 'BorrowerState'] = 'AR'
fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '08', 'BorrowerState'] = 'CO'
fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '09', 'BorrowerState'] = 'CT'
fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '11', 'BorrowerState'] = 'DC'
fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '12', 'BorrowerState'] = 'FL'
fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '20', 'BorrowerState'] = 'KS'
fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '22', 'BorrowerState'] = 'LA'
fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '24', 'BorrowerState'] = 'MD'
fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '35', 'BorrowerState'] = 'NM'
fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '36', 'BorrowerState'] = 'NY'
fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '37', 'BorrowerState'] = 'NC'
fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '40', 'BorrowerState'] = 'OK'
fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '47', 'BorrowerState'] = 'TN'
fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '48', 'BorrowerState'] = 'TX'
fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '50', 'BorrowerState'] = 'VT'
fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '51', 'BorrowerState'] = 'VA'


Check

Manual Update

In [23]:
fips_fill = pd.read_csv("../../../data/state_data/geo/ppp_geo_fips23.csv", index_col=0) 

fips_fill.loc[fips_fill["LoanNumber"]==5601547110, "full_add"]="1750 US-40, Vernal, UT"
fips_fill.loc[fips_fill["LoanNumber"]==5601547110, "Lat"]="40.412219"
fips_fill.loc[fips_fill["LoanNumber"]==5601547110, "Long"]="-109.494734"
fips_fill.loc[fips_fill["LoanNumber"]==5601547110, "FIPS_z"]="490479682012019"


data_xls = pd.read_excel('../../../data/null/tx_fips_update.xlsx', dtype=str, index_col=None)
data_xls.to_csv('../../../data/null/tx_fips_update.csv', encoding='utf-8', index=False)
tx_update_df = pd.read_csv("../../../data/null/tx_fips_update.csv").set_index(['index']).rename_axis(None)
tx_update_df.index = tx_update_df.index.astype('int64')
tx_update_df

Read in base data

In [47]:
#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/ppp_geo_fips23.csv", index_col=0) 
fips_check.head()

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,5502308207,AK,"5015 BUSINESS PARK BLVD, ANCHORAGE, AK",61.174949,-149.888309,2.020002e+13
1,6110847106,AK,"7000 Uula St, BARROW, AK",71.297290,-156.728897,2.185000e+13
2,4539098204,AK,"4700 Old International Airport Rd, Anchorage, AK",61.173232,-149.972718,2.020002e+13
3,5120868804,AK,"540 W Intl Airport Rd, Anchorage, AK",61.173164,-149.893283,2.020003e+13
4,6650277102,AK,"4300 BARTLETT STREET, HOMER, AK",59.652704,-151.549793,2.122001e+13


In [18]:
fips_check.loc[fips_check["LoanNumber"]==6790008405]

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
938437,6790008405,WA,"515 KUWILI ST UNIT E, HONOLULU, WA",21.317822,-157.868017,NaN


Update

In [48]:
fips_check.loc[fips_check.LoanNumber.isin(fips_fill.LoanNumber), ['BorrowerState', 'full_add', 'Lat', 'Long', 'FIPS_z']] = fips_fill[['BorrowerState', 'full_add', 'Lat', 'Long', 'FIPS_z']].values
fips_check

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,5502308207,AK,"5015 BUSINESS PARK BLVD, ANCHORAGE, AK",61.174949,-149.888309,20200019003022.0
1,6110847106,AK,"7000 Uula St, BARROW, AK",71.297290,-156.728897,21850001001069.0
2,4539098204,AK,"4700 Old International Airport Rd, Anchorage, AK",61.173232,-149.972718,20200023011009.0
3,5120868804,AK,"540 W Intl Airport Rd, Anchorage, AK",61.173164,-149.893283,20200025012000.0
4,6650277102,AK,"4300 BARTLETT STREET, HOMER, AK",59.652704,-151.549793,21220010002001.0
...,...,...,...,...,...,...
965547,9725917702,Not Available,"17 S High St., COLUMBUS, OH",39.961717,-83.001011,NaN
965548,9666867710,Not Available,"17 S High St., COLUMBUS, OH",39.961717,-83.001011,NaN
965549,9662697710,Not Available,"17 S High St., COLUMBUS, OH",39.961717,-83.001011,NaN
965550,9724667708,Not Available,"17 S High St., COLUMBUS, OH",39.961717,-83.001011,NaN


Verify

In [50]:
fips_check.loc[fips_check["LoanNumber"]==5924068305]

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
318097,5924068305,IL,"43W450 US-30, Sugar Grove, IL",41.768316,-88.462704,170898545041040


Write to MainFrame

In [51]:
fips_check.to_csv("../../../data/state_data/geo/ppp_geo_fips23.csv") 

Run through fipssplit after write to update state Files